# Fake NEws Detection


In [53]:
#importing files/modules
import numpy as np
import pandas as pd
import gradio as gr
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

In [54]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
#printing stop words
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [56]:
# Loading the Dataset into pandas Dataframe
news_dataset = pd.read_csv(r'C:\Users\Dell\Desktop\miniproject\train.csv')


In [57]:
news_dataset = news_dataset.fillna('')
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']
labels=news_dataset.label
print(X)
print(Y)



          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [58]:
news_dataset.head()


,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [59]:
news_dataset.isnull().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [60]:
news_dataset = news_dataset.fillna('')

In [61]:
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [62]:
# Seperating the data and Label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [63]:
port_stem = PorterStemmer()

In [64]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()   
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] 
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [65]:
news_dataset['content'] =  news_dataset['content'].apply(stemming)

In [66]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [67]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state=2)

In [68]:
# transform the text data to feature vectors that can be used as input to the Logistic regression

feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase='True')

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# convert Y_train and Y_test values as integers

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [69]:
model = LogisticRegression()

In [70]:
model.fit(X_train_features, Y_train)

LogisticRegression()

In [71]:
# accuracy score on the training data set
X_train_prediction = model.predict(X_train_features)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [72]:
print('Accuracy Score of our training data :', training_data_accuracy)

Accuracy Score of our training data : 0.9867788461538461


In [73]:
# accuracy score on test data set
X_test_prediction = model.predict(X_test_features)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [74]:
input_mail = ["Facebook Wall: Reactions to Donald Trumpâ€™s victory in US Presidential elections	UnReal Mama"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_mail)

print(input_data_features)

# making prediction

prediction = model.predict(input_data_features)
print(prediction)

if(prediction[0] == 0):
    print("The news is Real")
else:
    print("The news is Fake")

  (0, 14740)	0.3652178226419815
  (0, 14328)	0.5339526278728929
  (0, 8170)	0.6101935480620226
  (0, 4598)	0.38272564054306657
  (0, 3834)	0.2503905972409137
[1]
The news is Fake


In [75]:
def news_check(input_author, input_title):
    # convert text to feature vectors
    input_mail = [input_title + ' ' + input_author]
    input_data_features = feature_extraction.transform(input_mail)

    # making prediction

    prediction = model.predict(input_data_features)

    if(prediction[0] == 0):
        return "The news is Real!"
    else:
        return "The news is Fake..."

In [76]:
filename ='tfidf.pkl'
pickle.dump(feature_extraction,open(filename,'wb'))